In [2]:
!apt-get update -y
!apt-get install -y chromium-chromedriver
!pip install -q selenium pandas openpyxl


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [357 B]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [70.9 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [4,115 kB]
Get:13 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.2 kB]

In [3]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=15d44fac7dcfcae5661d213db6fb15a740a064002cdc366cecde8f1297765b17
  Stored in directory: /root/.cache/pip/wheels/0c/47/f5/89b7e770ab2996baf8c910e7353d6391e373075a0ac213519e
Successfully built google-search-results


In [6]:
from serpapi import GoogleSearch
import pandas as pd
import time

api_key = "783dcf17ea65c6a84a44cceb4fd71c5129f14f0eefc1d4cfc5c3249f8a095bd4"

queries = [
    # Makkah
    "Hotels in Makkah",
    "Hotels near Masjid al Haram",
    "Hotels in Ajyad Makkah",
    "Hotels in Aziziyah Makkah",
    "Hotels in Al Misfalah Makkah",
    "Hotels in Al Shubaikah Makkah",
    "Hotels in Al Naseem Makkah",
    "Hotels in Kudai Makkah",
    "Hotels in Jarwal Makkah",

    # Madinah
    "Hotels in Madinah",
    "Hotels near Al Masjid an Nabawi",
    "Hotels in Central Madinah",
    "Hotels in Al Haram Madinah",
    "Hotels in Quba Madinah",
    "Hotels in Al Azhari Madinah",
    "Hotels in Al Aridh Madinah",
    "Hotels in Bani Khidrah Madinah"
]

date_ranges = [
    ("2026-02-15", "2026-02-20"),
    ("2026-03-10", "2026-03-15"),
    ("2026-03-20", "2026-03-25"),
    ("2026-05-20", "2026-05-25"),
    ("2026-06-10", "2026-06-15"),
    ("2026-07-01", "2026-07-06")
]

all_hotels = []

for checkin, checkout in date_ranges:
    for q in queries:
        for start in [0, 20, 40, 60, 80]:

            params = {
                "engine": "google_hotels",
                "q": q,
                "check_in_date": checkin,
                "check_out_date": checkout,
                "adults": 2,
                "currency": "SAR",
                "gl": "sa",
                "hl": "ar",
                "start": start,
                "api_key": api_key
            }

            search = GoogleSearch(params)
            results = search.get_dict()

            for h in results.get("properties", []):


                if checkin.startswith("2026-03"):
                    season_type = "Ramadan Season"
                    context_text = "High spiritual tourism season during Ramadan"
                elif checkin.startswith("2026-06"):
                    season_type = "Hajj Peak"
                    context_text = "Peak pilgrimage demand period"
                elif checkin.startswith("2026-05"):
                    season_type = "Pre-Hajj"
                    context_text = "Pre-Hajj booking increase period"
                else:
                    season_type = "Regular Umrah"
                    context_text = "Regular travel season with moderate hotel demand"

                address = h.get("address")
                if not address:
                    gps = h.get("gps_coordinates", {})
                    lat = gps.get("latitude")
                    lng = gps.get("longitude")
                    if lat and lng:
                        address = f"Lat: {lat}, Lng: {lng}"
                    else:
                        address = None


                city = "Madinah" if "Madinah" in q else "Makkah"

                all_hotels.append({
                    "Hotel Name": h.get("name"),
                    "Price (SAR)": h.get("rate_per_night", {}).get("lowest"),
                    "Rating": h.get("overall_rating"),
                    "Reviews": h.get("reviews"),
                    "Address": address,
                    "City": city,
                    "Check-in": checkin,
                    "Check-out": checkout,
                    "Search Area": q,
                    "Season_Type": season_type,
                    "Religious_Context": context_text
                })

            time.sleep(1)

df_final = pd.DataFrame(all_hotels)

df_final = df_final.drop_duplicates(
    subset=["Hotel Name", "Address", "Check-in"]
).reset_index(drop=True)

print("Total rows collected:", len(df_final))

df_final.to_csv("makkah_madinah_hajj_umrah_2026_with_unstructured.csv", index=False)

df_final.head()


Total rows collected: 1137


,Hotel Name,Price (SAR),Rating,Reviews,Address,City,Check-in,Check-out,Search Area,Season_Type,Religious_Context
0,نجوم السعد 3,‏٨٦٤ ر.س.‏,NaN,NaN,"Lat: 21.372940063476562, Lng: 39.841705322265625",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
1,نجوم السعد 6,‏٩٠٢ ر.س.‏,2.2,3.0,"Lat: 21.372928619384766, Lng: 39.841705322265625",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
2,ضيافة الطلائع للشقق المخدومة - One-Bedroom Apa...,‏٣٬٣٠٠ ر.س.‏,NaN,NaN,"Lat: 21.422340393066406, Lng: 39.848140716552734",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
3,فندق الميقات اجياد,None,3.7,387.0,"Lat: 21.4197991, Lng: 39.828957599999995",Makkah,2026-03-10,2026-03-15,Hotels near Masjid al Haram,Ramadan Season,High spiritual tourism season during Ramadan
4,Hotel mekkah,‏٧٣١ ر.س.‏,5.0,3.0,"Lat: 21.418908599999998, Lng: 39.8290042",Makkah,2026-03-10,2026-03-15,Hotels near Masjid al Haram,Ramadan Season,High spiritual tourism season during Ramadan


In [7]:
from serpapi import GoogleSearch
import pandas as pd
import time

api_key = "783dcf17ea65c6a84a44cceb4fd71c5129f14f0eefc1d4cfc5c3249f8a095bd4"

queries = [
    "Hotels in Taif",
    "Hotels in Taif for Hajj visitors",
    "Hotels in Taif near Miqat",
    "Umrah visitor hotels in Taif",
    "Budget hotels in Taif for pilgrims",
    "Luxury hotels in Taif during Hajj season",
    "Hotels in Taif near Al Sail Al Kabir Miqat"
]

date_ranges = [
    ("2026-02-15", "2026-02-20"),
    ("2026-03-10", "2026-03-15"),
    ("2026-03-20", "2026-03-25"),
    ("2026-05-20", "2026-05-25"),
    ("2026-06-10", "2026-06-15"),
    ("2026-07-01", "2026-07-06")
]

all_hotels = []

for checkin, checkout in date_ranges:
    for q in queries:
        for start in [0, 20, 40]:

            params = {
                "engine": "google_hotels",
                "q": q,
                "check_in_date": checkin,
                "check_out_date": checkout,
                "adults": 2,
                "currency": "SAR",
                "gl": "sa",
                "hl": "ar",
                "start": start,
                "api_key": api_key
            }

            search = GoogleSearch(params)
            results = search.get_dict()

            for h in results.get("properties", []):


                if checkin.startswith("2026-03"):
                    season_type = "Ramadan Season"
                    context_text = "Ramadan spiritual tourism season affecting Taif hotels"
                elif checkin.startswith("2026-06"):
                    season_type = "Hajj Peak"
                    context_text = "Hajj season overflow demand with pilgrims staying in Taif"
                elif checkin.startswith("2026-05"):
                    season_type = "Pre-Hajj"
                    context_text = "Pre-Hajj increasing accommodation demand in Taif"
                else:
                    season_type = "Regular Umrah"
                    context_text = "Regular Umrah related travel near Makkah region"

                address = h.get("address")
                if not address:
                    gps = h.get("gps_coordinates", {})
                    lat = gps.get("latitude")
                    lng = gps.get("longitude")
                    if lat and lng:
                        address = f"Lat: {lat}, Lng: {lng}"
                    else:
                        address = None

                all_hotels.append({
                    "Hotel Name": h.get("name"),
                    "City": "Taif",
                    "Price (SAR)": h.get("rate_per_night", {}).get("lowest"),
                    "Rating": h.get("overall_rating"),
                    "Reviews": h.get("reviews"),
                    "Address": address,
                    "Check-in": checkin,
                    "Check-out": checkout,
                    "Search Area": q,
                    "Season_Type": season_type,
                    "Religious_Context": context_text
                })

            time.sleep(1)

df_taif = pd.DataFrame(all_hotels)

df_taif = df_taif.drop_duplicates(
    subset=["Hotel Name", "Address", "Check-in"]
).reset_index(drop=True)

print("Total Taif rows:", len(df_taif))

df_taif.to_csv("taif_hajj_umrah_2026.csv", index=False)

df_taif.head()

Total Taif rows: 323


,Hotel Name,City,Price (SAR),Rating,Reviews,Address,Check-in,Check-out,Search Area,Season_Type,Religious_Context
0,فندق وشاليهات الخزامى السياحي,Taif,‏١٩٢ ر.س.‏,3.8,1099.0,"Lat: 21.300456, Lng: 40.4335465",2026-03-10,2026-03-15,Hotels in Taif,Ramadan Season,Ramadan spiritual tourism season affecting Tai...
1,فندق كيان الطائف,Taif,‏٢٧١ ر.س.‏,4.3,1046.0,"Lat: 21.2944004, Lng: 40.4350932",2026-03-10,2026-03-15,Hotels in Taif,Ramadan Season,Ramadan spiritual tourism season affecting Tai...
2,سديم الفجر للشقق الفندقية - Sadeem Alfajr Hote...,Taif,‏٢٠٠ ر.س.‏,3.6,843.0,"Lat: 21.2882806, Lng: 40.416078500000005",2026-03-10,2026-03-15,Hotels in Taif,Ramadan Season,Ramadan spiritual tourism season affecting Tai...
3,سفن جاردنز-البيعة,Taif,‏٣٥٠ ر.س.‏,4.4,1584.0,"Lat: 21.284342499999998, Lng: 40.4451623",2026-03-10,2026-03-15,Hotels in Taif,Ramadan Season,Ramadan spiritual tourism season affecting Tai...
4,توت هاوس الوسام,Taif,‏٢٢٣ ر.س.‏,4.4,1126.0,"Lat: 21.233142600000004, Lng: 40.3731541",2026-03-10,2026-03-15,Hotels in Taif,Ramadan Season,Ramadan spiritual tourism season affecting Tai...


In [8]:
import pandas as pd

df_main = pd.read_csv("makkah_madinah_hajj_umrah_2026_with_unstructured.csv")
df_taif = pd.read_csv("taif_hajj_umrah_2026.csv")

print("Rows before merge:")
print("Main file:", len(df_main))
print("Taif file:", len(df_taif))

df_main.columns = df_main.columns.str.strip()
df_taif.columns = df_taif.columns.str.strip()

df_combined = pd.concat([df_main, df_taif], ignore_index=True, sort=False)

df_combined = df_combined.drop_duplicates(
    subset=["Hotel Name", "Address", "Check-in"]
).reset_index(drop=True)

print("Final merged rows:", len(df_combined))

df_combined.to_csv("FINAL_HAJJ_UMRAH_ALL_CITIES_2026.csv", index=False)

df_combined.head()


Rows before merge:
Main file: 1137
Taif file: 323
Final merged rows: 1460


,Hotel Name,Price (SAR),Rating,Reviews,Address,City,Check-in,Check-out,Search Area,Season_Type,Religious_Context
0,نجوم السعد 3,‏٨٦٤ ر.س.‏,NaN,NaN,"Lat: 21.372940063476562, Lng: 39.841705322265625",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
1,نجوم السعد 6,‏٩٠٢ ر.س.‏,2.2,3.0,"Lat: 21.372928619384766, Lng: 39.841705322265625",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
2,ضيافة الطلائع للشقق المخدومة - One-Bedroom Apa...,‏٣٬٣٠٠ ر.س.‏,NaN,NaN,"Lat: 21.422340393066406, Lng: 39.848140716552734",Makkah,2026-03-10,2026-03-15,Hotels in Makkah,Ramadan Season,High spiritual tourism season during Ramadan
3,فندق الميقات اجياد,NaN,3.7,387.0,"Lat: 21.4197991, Lng: 39.828957599999995",Makkah,2026-03-10,2026-03-15,Hotels near Masjid al Haram,Ramadan Season,High spiritual tourism season during Ramadan
4,Hotel mekkah,‏٧٣١ ر.س.‏,5.0,3.0,"Lat: 21.418908599999998, Lng: 39.8290042",Makkah,2026-03-10,2026-03-15,Hotels near Masjid al Haram,Ramadan Season,High spiritual tourism season during Ramadan
